In [ ]:
!pip install datasets
!pip install transformers
!pip install nltk
!git clone https://github.com/Yale-LILY/QMSum.git
!pip install sacrebleu
!pip install --upgrade accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'QMSum' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json

split = 'train'
data_path = 'QMSum/data/ALL/jsonl/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
# n_meetings = len(data)
# print('Total {} meetings in the {} set.'.format(n_meetings, split))

In [ ]:
len1 = [len(data[i]['specific_query_list']) for i in range(len(data))]

In [ ]:
sum(len1)

1095

In [ ]:
from nltk import word_tokenize

def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [ ]:
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [ ]:
import nltk
nltk.download('punkt')

bart_data_gold = []
for i in range(len(data)):
    entire_src = []
    for k in range(len(data[i]['meeting_transcripts'])):
        cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
        cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
        entire_src.append(cur_turn)
    entire_src = ' '.join(entire_src)
    for j in range(len(data[i]['general_query_list'])):
        cur = {}
        query = tokenize(data[i]['general_query_list'][j]['query'])
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + entire_src + ' </s>')
        target = tokenize(data[i]['general_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
    for j in range(len(data[i]['specific_query_list'])):
        cur = {}
        query = tokenize(data[i]['specific_query_list'][j]['query'])
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + entire_src + ' </s>')
        target = tokenize(data[i]['specific_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)

print('Total {} query-summary pairs in the {} set'.format(len(bart_data_gold), split))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Total 1257 query-summary pairs in the train set


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer,AutoConfig,LEDConfig
from transformers import AutoModelForSeq2SeqLM,LEDForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("pszemraj/led-large-book-summary")
model = AutoModelForSeq2SeqLM.from_pretrained("pszemraj/led-large-book-summary")

In [ ]:
from datasets import DatasetDict
from datasets import Dataset

dataset_dict = DatasetDict({
    "train": Dataset.from_dict(dict(zip(bart_data_gold[0].keys(), zip(*[d.values() for d in bart_data_gold]))))
})

In [ ]:
split_datasets = dataset_dict["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples['src'], truncation=True)
    labels = tokenizer(examples['tgt'], truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
)

Map:   0%|          | 0/1131 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets.push_to_hub(repo_id='tokenized_datasets_witout_cut', private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import DatasetDict
from datasets import Dataset
from datasets import load_dataset
tokenized_datasets = load_dataset('fgiuhsdfkjhfv/tokenized_datasets_witout_cut', use_auth_token=True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"longsec_withno_cut",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=5,
    predict_with_generate=True,
    tf32=True,
    gradient_checkpointing=True,
    push_to_hub=True,
    gradient_accumulation_steps=8,
    optim="adafactor",
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,)

/content/longsec_withno_cut is already a clone of https://huggingface.co/fgiuhsdfkjhfv/longsec_withno_cut. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=350, training_loss=2.3257156808035715, metrics={'train_runtime': 12693.786, 'train_samples_per_second': 0.445, 'train_steps_per_second': 0.028, 'total_flos': 1.766542942432297e+17, 'train_loss': 2.3257156808035715, 'epoch': 4.95})

In [ ]:
trainer.push_to_hub(tags="text2text", commit_message="Training complete") # อัพขึ้น hugging face

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/1.71G [00:00<?, ?B/s]

Upload file runs/Jun05_09-48-47_3060afc830c3/events.out.tfevents.1685958532.3060afc830c3.4573.0:   0%|        …

To https://huggingface.co/fgiuhsdfkjhfv/longsec_withno_cut
   4a97633..907898c  main -> main

   4a97633..907898c  main -> main

To https://huggingface.co/fgiuhsdfkjhfv/longsec_withno_cut
   907898c..772728a  main -> main

   907898c..772728a  main -> main



'https://huggingface.co/fgiuhsdfkjhfv/longsec_withno_cut/commit/907898c3b47275c14dd136f944470e53b667d167'